In [294]:


# imports
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
import requests
import urllib.parse
import os

import sqlite3
from sqlite3 import Error
import sqlalchemy as sqla
from sqlalchemy import create_engine


In [295]:
with open(f"{os.path.expanduser('~')}/temp-keys/fsq-primary") as f:
    fsq_key = f.readline().rstrip()

with open(f"{os.path.expanduser('~')}/temp-keys/yelp-key") as f:
    yelp_key = f.readline().rstrip()

# Get location coordinates using nominatim api


In [296]:
# the_place = 'Pomona College'
the_place = '333 N College Way, Claremont CA 91711 '
# address = input("Please enter a location to query!")

def get_coords_str(place_name_or_address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(place_name_or_address) +'?format=json'

    response = requests.get(url).json()
    latitude = response[0]['lat']
    longitude = response[0]['lon']
    return latitude + "," + longitude

def get_coords_tuple(place_name_or_address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(place_name_or_address) +'?format=json'

    response = requests.get(url).json()
    latitude = response[0]['lat']
    longitude = response[0]['lon']
    return latitude,longitude

radius = 1000
limit = 50
fsq_categoryid = 13065

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [297]:
def fsq_place_query(place):
    loc_param = "ll=" + place
    rad_param= "radius=" + str(radius)
    cat_param = "categories=" + str(fsq_categoryid)
    fsq_url = f"https://api.foursquare.com/v3/places/search?"
    fields_param="&fields=fsq_id%2Cname%2Crating%2Ccategories%2Clocation%2Cdistance"
    limit_param=f"limit={limit}"
    headers = {
    "Accept": "application/json",
    "Authorization": fsq_key
    }
    return requests.get(fsq_url + loc_param + "&" +cat_param + "&" + rad_param + fields_param + "&" +limit_param, headers=headers)


Parse through the response to get the POI details you want (rating, name, location, etc)

In [298]:
with fsq_place_query(get_coords_str(the_place)) as fsq_response:
    fsq_data = json.loads(fsq_response.text)

# parsed = fsq_response.json()["results"]
# pd.read_json(fsq_response.text)


fsq_data

{'results': [{'fsq_id': '4a11eb6ff964a5205e771fe3',
   'categories': [{'id': 13002,
     'name': 'Bakery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'distance': 629,
   'location': {'address': '119 Yale Ave',
    'address_extended': '# 4723',
    'census_block': '060374019021048',
    'country': 'US',
    'cross_street': '1st St.',
    'dma': 'Los Angeles',
    'formatted_address': '119 Yale Ave (1st St.), Claremont, CA 91711',
    'locality': 'Claremont',
    'neighborhood': ['North Pomona'],
    'postcode': '91711',
    'region': 'CA'},
   'name': 'Some Crust Bakery',
   'rating': 9.1},
  {'fsq_id': '532a4a9c498e372406740e49',
   'categories': [{'id': 13068,
     'name': 'American Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_

Put your parsed results into a DataFrame

In [299]:
fsq_df_context = json_normalize(fsq_data['context'])
# fsq_df_main = json_normalize(fsq_data,"results",meta=[['context','geo_bounds','circle',],['context','geo_bounds','circle','center']],errors='ignore')
fsq_df_main = json_normalize(fsq_data,"results",errors='ignore')
fsq_df_category = json_normalize(fsq_data['results'],"categories",meta=["fsq_id","name"], meta_prefix='place_', errors='ignore')

fsq_df_main


,fsq_id,categories,distance,name,rating,location.address,location.address_extended,location.census_block,location.country,location.cross_street,location.dma,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.po_box
0,4a11eb6ff964a5205e771fe3,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",629,Some Crust Bakery,9.1,119 Yale Ave,# 4723,060374019021048,US,1st St.,Los Angeles,"119 Yale Ave (1st St.), Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
1,532a4a9c498e372406740e49,"[{'id': 13068, 'name': 'American Restaurant', ...",464,Bardot,8.5,206 W Bonita Ave,NaN,060374019021037,US,,Los Angeles,"206 W Bonita Ave, Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
2,4bedbc63946c0f473dd9a263,"[{'id': 13046, 'name': 'Ice Cream Parlor', 'ic...",557,Bert & Rocky's Cream Company,8.7,242 Yale Ave,NaN,060374019021044,US,Bonita Ave,Los Angeles,"242 Yale Ave (Bonita Ave), Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
3,5ba512302b274a002cdcaf8f,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",501,Crème Bakery,8.5,116 Harvard Ave,NaN,060374019021041,US,,Los Angeles,"116 Harvard Ave, Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
4,51ad311d498e888a24801b56,"[{'id': 13306, 'name': 'Taco Restaurant', 'ico...",540,Petiscos,8.3,211 W 1st St,NaN,060374019021042,US,Harvard Ave,Los Angeles,"211 W 1st St (Harvard Ave), Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
5,4c2cdcac77cfe21e08b2b6f1,"[{'id': 13302, 'name': 'Mediterranean Restaura...",534,Saca's Mediterranean Cuisine,8.2,248 W 2nd St,NaN,060374019021042,US,,Los Angeles,"248 W 2nd St, Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
6,4ba2568cf964a520bdee37e3,"[{'id': 13001, 'name': 'Bagel Shop', 'icon': {...",596,42nd Street Bagel,8.2,225 Yale Ave,Ste A,060374019021045,US,,Los Angeles,"225 Yale Ave, Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
7,4daa3f96a86e771ea720622f,"[{'id': 13236, 'name': 'Italian Restaurant', '...",689,Uno Tre Otto,8.3,114 N Indian Hill Blvd,Ste P,060374019021047,US,NaN,Los Angeles,"114 N Indian Hill Blvd, Claremont, CA 91711",Claremont,[North Pomona],91711,CA,NaN
8,4daa27184df01c19b1842ac1,"[{'id': 13065, 'name': 'Restaurant', 'icon': {...",979,Eureka! Claremont,8.8,580 W 1st St,NaN,060374019022012,US,at Cornell Ave,Los Angeles,"580 W 1st St (at Cornell Ave), Claremont, CA 9...",Claremont,[North Pomona],91711,CA,NaN
9,4b78649cf964a520e7ca2ee3,"[{'id': 13066, 'name': 'Afghan Restaurant', 'i...",560,Walter's Restaurant,8.0,310 Yale Ave,NaN,060374019021035,US,btwn 4th & Bonita,Los Angeles,"310 Yale Ave (btwn 4th & Bonita), Claremont, C...",Claremont,[North Pomona],91711,CA,P.O. Box 453


# Yelp

Send a request with the same location parameters (location, radius, etc)

In [300]:
def yelp_place_query(place):
    latitude,longitude = get_coords_tuple(place)
    yelp_url = "https://api.yelp.com/v3/businesses/search?"
    yelp_params = f"latitude={latitude}&longitude={longitude}&radius={radius}&categories=restaurants, all&limit={limit}"

    # print(yelp_url + yelp_params)
    headers = {
        "Authorization": f"Bearer {yelp_key}"
    }
    return requests.get(yelp_url + yelp_params, headers=headers)


yelp_res = yelp_place_query(the_place)


Parse through your result and get POI details

In [301]:
yelp_data = json.loads(yelp_res.text)
yelp_df_main = json_normalize(yelp_data,"businesses")
yelp_df_categories = json_normalize(yelp_data["businesses"],"categories",meta=['id','name'])

Put your parsed results into a DataFrame

In [302]:
yelp_df_main["location.display_address"] = yelp_df_main["location.display_address"].map(lambda x: str(x).replace('[',"").replace("]","").replace("'",""))
yelp_df_main

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,SlhHQiSpqxOk8BzCtdjK4Q,dr-grubbs-claremont-2,Dr Grubb's,https://s3-media1.fl.yelpcdn.com/bphoto/VzSUeP...,False,https://www.yelp.com/biz/dr-grubbs-claremont-2...,1139,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,[delivery],...,34.096809,-117.718754,373 W Bonita Ave,None,None,Claremont,91711,US,CA,"373 W Bonita Ave, Claremont, CA 91711"
1,FzX6_0QFfsKFuJzys6qyPQ,the-quarter-creole-cuisine-claremont-3,The Quarter Creole Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/H17Xn8...,False,https://www.yelp.com/biz/the-quarter-creole-cu...,762,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.5,"[delivery, pickup, restaurant_reservation]",...,34.095830,-117.719620,201 N Indian Hill Blvd,Ste 103,None,Claremont,91711,US,CA,"201 N Indian Hill Blvd, Ste 103, Claremont, CA..."
2,BpQkiZJAqoFGy1Ysd6hZzg,some-crust-bakery-claremont,Some Crust Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/F5hRDP...,False,https://www.yelp.com/biz/some-crust-bakery-cla...,1243,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,[delivery],...,34.095078,-117.718178,119 Yale Ave,,,Claremont,91711,US,CA,"119 Yale Ave, Claremont, CA 91711"
3,TFxcmjVIsW5bdW7rilHHpQ,eureka-claremont-3,Eureka! Claremont,https://s3-media1.fl.yelpcdn.com/bphoto/7W9g1x...,False,https://www.yelp.com/biz/eureka-claremont-3?ad...,2954,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"[delivery, pickup]",...,34.094379,-117.721687,580 W 1st St,,,Claremont,91711,US,CA,"580 W 1st St, Claremont, CA 91711"
4,JLyP3zNKUb1VFD0MVj0ZxQ,the-last-drop-cafe-claremont,The Last Drop Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/4JIg5I...,False,https://www.yelp.com/biz/the-last-drop-cafe-cl...,697,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"[delivery, pickup]",...,34.095020,-117.716428,119 Harvard Ave,None,,Claremont,91711,US,CA,"119 Harvard Ave, Claremont, CA 91711"
5,JlN5BhjbErkp6myx-8Adkw,kazama-sushi-claremont,Kazama Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/7mJuFD...,False,https://www.yelp.com/biz/kazama-sushi-claremon...,1061,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,[delivery],...,34.095166,-117.719514,101 N Indian Hill Blvd,Ste C1-104,,Claremont,91711,US,CA,"101 N Indian Hill Blvd, Ste C1-104, Claremont,..."
6,81IjU5L-t-QQwsE38C63hQ,the-back-abbey-claremont,The Back Abbey,https://s3-media2.fl.yelpcdn.com/bphoto/3x0j0D...,False,https://www.yelp.com/biz/the-back-abbey-clarem...,2060,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,[delivery],...,34.094939,-117.720535,128 N Oberlin Ave,,,Claremont,91711,US,CA,"128 N Oberlin Ave, Claremont, CA 91711"
7,LyxuqM3_4t21MPRPSaWN1A,the-whisper-house-claremont,The Whisper House,https://s3-media1.fl.yelpcdn.com/bphoto/65NFXM...,False,https://www.yelp.com/biz/the-whisper-house-cla...,1754,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"[delivery, restaurant_reservation]",...,34.094573,-117.720321,502 W 1st St,,,Claremont,91711,US,CA,"502 W 1st St, Claremont, CA 91711"
8,LWN_UEueFvwB7whooBfPjA,bardot-claremont-2,Bardot,https://s3-media2.fl.yelpcdn.com/bphoto/hgLumF...,False,https://www.yelp.com/biz/bardot-claremont-2?ad...,1314,"[{'alias': 'newamerican', 'title': 'American (...",4.0,[delivery],...,34.096527,-117.716716,206 W Bonita Ave,,,Claremont,91711,US,CA,"206 W Bonita Ave, Claremont, CA 91711"
9,ynhSJscAI1aumkgEBUsO4A,viva-madrid-claremont,Viva Madrid!,https://s3-media2.fl.yelpcdn.com/bphoto/xGnCjj...,False,https://www.yelp.com/biz/viva-madrid-claremont...,1415,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",4.0,[delivery],...,34.096200,-117.718290,225 Yale Ave,Ste B,,Claremont,91711,US,CA,"225 Yale Ave, Ste B, Claremont, CA 91711"


In [303]:
yelp_df_categories


,alias,title,id,name
0,tradamerican,American (Traditional),SlhHQiSpqxOk8BzCtdjK4Q,Dr Grubb's
1,cajun,Cajun/Creole,FzX6_0QFfsKFuJzys6qyPQ,The Quarter Creole Cuisine
2,bakeries,Bakeries,BpQkiZJAqoFGy1Ysd6hZzg,Some Crust Bakery
3,breakfast_brunch,Breakfast & Brunch,BpQkiZJAqoFGy1Ysd6hZzg,Some Crust Bakery
4,customcakes,Custom Cakes,BpQkiZJAqoFGy1Ysd6hZzg,Some Crust Bakery
...,...,...,...,...
114,italian,Italian,esEFM18aHAgmgLQ1E4WxuQ,Pellegrino's Authentic Italian Food
115,cafes,Cafes,ij6NzMhxhtfsdefJnVZIHQ,Norema
116,coffee,Coffee & Tea,ij6NzMhxhtfsdefJnVZIHQ,Norema
117,coffee,Coffee & Tea,zTSr3T3BkjrfdIDRTLaJuw,Honnold Cafe


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [306]:

db_path = ("../data/places_local_db.db")

def create_conn_engine(path):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        engine = create_engine(f'sqlite:///{path}',echo=False)
    except Error as e:
        print(e)
    finally:
        if engine:
            return engine

the_engine = create_conn_engine(db_path)
fsq_df_main_2 = fsq_df_main.copy()
fsq_df_main_2.drop('categories',axis=1,inplace=True)
# fsq_df_main_2.drop('chains',axis=1,inplace=True)
fsq_df_main_2.drop('location.neighborhood',axis=1,inplace=True)


yelp_df_main_2 = yelp_df_main.copy()
yelp_df_main_2.drop('categories',axis=1,inplace=True)
yelp_df_main_2.drop('transactions',axis=1,inplace=True)


try:
    fsq_df_main_2.to_sql('fsq_main',con=the_engine, if_exists='replace')
    fsq_df_category.to_sql('fsq_category',con=the_engine,if_exists='replace')
    yelp_df_main_2.to_sql('yelp_main',con=the_engine,if_exists='replace')
    yelp_df_categories.to_sql('yelp_category',con=the_engine,if_exists='replace')
except Error as e:
    print(e)
finally:
    print("did it work?")


did it work?


Get the top 10 restaurants according to their rating

In [307]:
# the_engine.table_names()
with the_engine.connect() as conn:
    the_fsq = sqla.Table('yelp_category',sqla.MetaData(),autoload=True,autoload_with=the_engine)

print(the_fsq.columns)

ImmutableColumnCollection(yelp_category.index, yelp_category.alias, yelp_category.title, yelp_category.id, yelp_category.name)


In [308]:
yelp_top_ten_query = """
SELECT
fsq_main.fsq_id AS id, fsq_main.name AS name ,fsq_main.rating AS rating, fsq_category.category AS category
FROM fsq_main
JOIN fsq_category ON fsq_main.fsq_id = fsq_category.place_fsq_id
WHERE fsq_category.category LIKE '%Restaurant%'
GROUP BY fsq_main.fsq_id
ORDER BY fsq_main.rating
LIMIT 10
"""

In [309]:
# %load_ext sql
# %sql sqlite:///../data/places_local_db.db

In [310]:
%%sql
SELECT
fsq_main.fsq_id AS id, fsq_main.name AS name ,fsq_main.rating AS rating, fsq_category.name AS category
FROM fsq_main
JOIN fsq_category ON fsq_main.fsq_id = fsq_category.place_fsq_id
WHERE fsq_category.name LIKE '%Restaurant%'
GROUP BY fsq_main.fsq_id
ORDER BY fsq_main.rating DESC
LIMIT 10

 * sqlite:///../data/places_local_db.db
Done.


id,name,rating,category
4a11eb6ff964a5205e771fe3,Some Crust Bakery,9.1,Restaurant
4daa27184df01c19b1842ac1,Eureka! Claremont,8.8,Restaurant
4bedbc63946c0f473dd9a263,Bert & Rocky's Cream Company,8.7,Restaurant
5ba512302b274a002cdcaf8f,Crème Bakery,8.5,Restaurant
532a4a9c498e372406740e49,Bardot,8.5,American Restaurant
51ad311d498e888a24801b56,Petiscos,8.3,Taco Restaurant
4daa3f96a86e771ea720622f,Uno Tre Otto,8.3,Italian Restaurant
4c2cdcac77cfe21e08b2b6f1,Saca's Mediterranean Cuisine,8.2,Mediterranean Restaurant
4ba2568cf964a520bdee37e3,42nd Street Bagel,8.2,Sandwich Restaurant
4b78649cf964a520e7ca2ee3,Walter's Restaurant,8.0,Afghan Restaurant


In [311]:
%%sql
SELECT
fsq_main.name AS name ,fsq_main.rating AS rating, fsq_category.name AS category, fsq_main.distance as distance
FROM fsq_main
JOIN fsq_category ON fsq_main.fsq_id = fsq_category.place_fsq_id
GROUP BY fsq_main.fsq_id
ORDER BY fsq_main.rating DESC
LIMIT 10

 * sqlite:///../data/places_local_db.db
Done.


name,rating,category,distance
Some Crust Bakery,9.1,Bakery,629
Eureka! Claremont,8.8,Restaurant,979
Bert & Rocky's Cream Company,8.7,Ice Cream Parlor,557
Crème Bakery,8.5,Bakery,501
Bardot,8.5,American Restaurant,464
The Back Abbey,8.4,Burger Joint,850
Petiscos,8.3,Taco Restaurant,540
Uno Tre Otto,8.3,Italian Restaurant,689
Gus's BBQ,8.2,BBQ Joint,911
Saca's Mediterranean Cuisine,8.2,Mediterranean Restaurant,534


In [315]:
%%sql
SELECT
yelp_main.name AS name, yelp_main.rating * 2 AS rating, yelp_category.title AS category, round(yelp_main.distance) AS 'distance (m)'
FROM yelp_main
JOIN yelp_category ON yelp_main.id = yelp_category.id
WHERE distance <=1000
GROUP BY yelp_main.id
ORDER BY yelp_main.rating DESC
LIMIT 10

 * sqlite:///../data/places_local_db.db
Done.


name,rating,category,distance (m)
Norema,10.0,Cafes,587.0
Chocolate Bash,9.0,Coffee & Tea,807.0
Crepes of Wrath,9.0,Creperies,541.0
The Claremont Village Eatery,9.0,Breakfast & Brunch,529.0
Oni Sushi,9.0,Poke,686.0
"Cafe X2O, Claremont",9.0,Mediterranean,807.0
Wings & Bowls,9.0,Chicken Wings,807.0
Dr Grubb's,9.0,American (Traditional),651.0
Kazama Sushi,9.0,Beer Bar,762.0
The Last Drop Cafe,9.0,Bakeries,512.0


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).